In [1]:
import pandas as pd
import re

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [2]:
ratings_movies = pd.read_csv('data/movies_data/ratings_movies.csv', sep=',')

In [5]:
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)

In [6]:
ratings_movies.head()

,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995.0
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995.0
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995.0
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995.0


In [31]:

print(ratings_movies[ratings_movies['year_release'].isna()]['year_release'].shape[0]) # фильмы у которых нет года выпуска



18


In [44]:
year1999_mask = ratings_movies['year_release'] == 1999
ratings_movies[year1999_mask].pivot_table(values='rating', index='title', aggfunc='mean').sort_values('rating') # какой фильм в 1999 году получил меньшую среднюю оценку

,rating
title,
Bloodsport: The Dark Kumite (1999),0.5
Simon Sez (1999),1.0
Chill Factor (1999),1.0
"Source, The (1999)",1.0
Trippin' (1999),1.0
...,...
Trailer Park Boys (1999),5.0
Larry David: Curb Your Enthusiasm (1999),5.0
Sun Alley (Sonnenallee) (1999),5.0


In [38]:
# какой фильм в 1999 году получил меньшую среднюю оценку. С помощью группировки
ratings_movies[year1999_mask].groupby(['title'])['rating'].mean().sort_values(ascending=False)

title
George Carlin: You Are All Diseased (1999)          5.0
Mickey's Once Upon a Christmas (1999)               5.0
Trailer Park Boys (1999)                            5.0
Larry David: Curb Your Enthusiasm (1999)            5.0
On the Ropes (1999)                                 5.0
                                                   ... 
Chill Factor (1999)                                 1.0
Simon Sez (1999)                                    1.0
Trippin' (1999)                                     1.0
From Dusk Till Dawn 2: Texas Blood Money (1999)     1.0
Bloodsport: The Dark Kumite (1999)                  0.5
Name: rating, Length: 261, dtype: float64

In [48]:
year2010_mask = ratings_movies['year_release'] == 2010
ratings_movies[year2010_mask].groupby(['genres'])['rating'].mean().sort_values(ascending=False)

genres
Animation|Drama|Fantasy|Mystery      5.000000
Animation|Children|Mystery           5.000000
Adventure|Children|Comedy|Mystery    5.000000
Comedy|Musical                       5.000000
Sci-Fi                               4.750000
                                       ...   
Adventure|Comedy|Fantasy             1.833333
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Action|Sci-Fi                        1.000000
Name: rating, Length: 119, dtype: float64

In [119]:
users_views = ratings_movies.groupby(['userId'])['genres'].agg(['nunique','count']).sort_values(by='nunique')
display(users_views) # количество просмотров уникальных жанров фильмов 


,nunique,count
userId,,
214,13,22
85,13,34
245,13,21
494,15,22
578,15,27
...,...,...
474,395,2108
380,399,1218
448,403,1864


In [127]:
# Найти пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая (ответ 1 строка)
ratings_movies.groupby('userId')['rating'].agg(['count','mean']).sort_values(by=['count', 'mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [131]:
year2018_mask = ratings_movies['year_release'] == 2018
ratings_movies_2018 = ratings_movies[year2018_mask]
ratings_movies_2018 = ratings_movies_2018.groupby('genres')['rating'].agg(['count', 'mean']).sort_values(by=['mean', 'count'], ascending=[False, True])
ratings_movies_2018[ratings_movies_2018['count'] > 10]

,count,mean
genres,,
Action|Adventure|Sci-Fi,14,3.928571
Action|Comedy|Sci-Fi,12,3.875000


In [134]:
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])
ratings_movies['year_rating'] = ratings_movies['date'].dt.year

In [153]:
ratings_movies.pivot_table(
    values='rating', 
    index='year_rating', 
    columns='genres',
    aggfunc='mean'
)['Comedy'].sort_values()

year_rating
2017    2.852668
2007    2.928187
2005    2.963325
1998    3.000000
2015    3.106183
2003    3.120066
2018    3.121296
2000    3.141291
2010    3.179825
2002    3.198556
1996    3.228571
2011    3.232877
2014    3.271429
2009    3.273292
2001    3.318408
2006    3.347534
2004    3.356877
2016    3.373431
1997    3.409091
2008    3.426667
2012    3.478477
2013    3.489474
1999    3.606061
Name: Comedy, dtype: float64

In [155]:
orders = pd.read_csv('data/orders_and_products/orders.csv', sep=';')
products = pd.read_csv('data/orders_and_products/products.csv', sep=';')

In [156]:
orders.info()
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Дата создания  18 non-null     object
 1   Order ID       18 non-null     int64 
 2   ID Покупателя  18 non-null     int64 
 3   Статус         18 non-null     object
 4   Оплачен        18 non-null     object
 5   Отменен        18 non-null     object
 6   Отгружен       18 non-null     object
 7   ID товара      18 non-null     int64 
 8   Количество     18 non-null     int64 
dtypes: int64(4), object(5)
memory usage: 1.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Product_ID  14 non-null     int64 
 1   Name        14 non-null     object
 2   Price       14 non-null     int64 
 3   CURRENCY    14 non-null     object
dtypes: int64(2), object(2)
memor

In [157]:
display(orders)
display(products)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR
5,86,"Носки Простые, муж",45,RUR
6,91,"Носки Честные, муж",50,RUR
7,103,"Носки Подарочные, муж",199,RUR
8,104,"Носки Подарочные, жен",249,RUR
9,124,Носки беговые Camino,999,RUR


In [159]:
orders_products = orders.merge(products, how='left', left_on='ID товара', right_on='Product_ID')
display(orders_products)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,91.0,"Носки Честные, муж",50.0,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,103.0,"Носки Подарочные, муж",199.0,RUR
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,104.0,"Носки Подарочные, жен",249.0,RUR


In [161]:
orders_products['profit'] = orders_products['Количество'] * orders_products['Price']

In [165]:
orders_products.groupby('ID Покупателя')['profit'].agg('sum')

ID Покупателя
1      2240.0
5     14042.0
7     17096.0
8      3087.0
9      7040.0
10      995.0
Name: profit, dtype: float64